In [ ]:
#Required packages

from meteostat import Point, Hourly
from datetime import datetime
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet

#Optional if you want to apply graphics, etc...

import seaborn as sns
import matplotlib.pyplot as plt

start = datetime(2018, 1, 1)
end = datetime(2023, 4, 18)
location = Point(35.9420, -95.8833) #Bixby, OK
data = Hourly(location, start, end)
data = data.normalize()
data = data.interpolate()
data = data.fetch()
dataset=data
data.to_csv('base_data')
table=pd.read_csv('base_data')

table['ds']=pd.DatetimeIndex(table['temp']) 
data1=table[['time','temp','dwpt', 'rhum', 'pres', 'prcp']]
data1.columns=['ds','y','dwpt','rhum','pres', 'prcp']
data1.head
data1.head
df_temp=data1
df_temp=df_temp.fillna(0)


m = NeuralProphet()
m = m.add_future_regressor('dwpt')
m = m.add_future_regressor('rhum')
m = m.add_future_regressor('pres')
m = m.add_future_regressor('prcp')

metrics = m.fit(df_temp, freq='H')
temp_forecast = m.predict(df_temp)


now = datetime.now()
future=(temp_forecast["ds"] > now)
temp_forecast['Future?'] = future
future_forecast = temp_forecast[temp_forecast['Future?'] == True]

In [ ]:
future_forecast.to_csv('future_forecast')

In [ ]:
heatmap=pd.read_csv('future_forecast', index_col=0) 
heatmap["InsertedDateTime"]= pd.to_datetime(heatmap["ds"])
heatmap['DayOfMonth']=heatmap["InsertedDateTime"].dt.day
heatmap['Hour']=heatmap["InsertedDateTime"].dt.hour
heatmap

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

# Load the example flights dataset and convert to long-form

outline = heatmap.pivot("Hour", "DayOfMonth",'yhat1')

# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(outline, annot=True, linewidths=.5, ax=ax, cmap="rainbow").figure.savefig("bixby.png")

In [ ]:
now